<a href="https://colab.research.google.com/github/anshkumar/elf/blob/master/elf_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!cp "./drive/My Drive/data.zip" .

In [39]:
!unzip ./data.zip

Archive:  ./data.zip
replace data/VIC/PRICE_AND_DEMAND_201501_VIC1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [40]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.2 GB  | Proc size: 1.3 GB
GPU RAM Free: 11172MB | Used: 269MB | Util   2% | Total 11441MB


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Oct  9 19:05:59 2018

@author: vedanshu
"""

from collections import UserList, UserDict
from typing import Union, Iterable, Tuple, Dict, Any

import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import os.path

def _meta_file(path):
    return os.path.join(path, 'feeder_meta.pkl')


class VarFeeder:
    """
    Builds temporary TF graph, injects variables into, and saves variables to TF checkpoint.
    In a train time, variables can be built by build_vars() and content restored by FeederVars.restore()
    """
    def __init__(self, path: str,
                 tensor_vars: Dict[str, Union[pd.DataFrame, pd.Series, np.ndarray]] = None,
                 plain_vars: Dict[str, Any] = None):
        """
        :param path: dir to store data
        :param tensor_vars: Variables to save as Tensors (pandas DataFrames/Series or numpy arrays)
        :param plain_vars: Variables to save as Python objects
        """
        tensor_vars = tensor_vars or dict()

        def get_values(v):
            v = v.values if hasattr(v, 'values') else v
            if not isinstance(v, np.ndarray):
                v = np.array(v)
            if v.dtype == np.float64:
                v = v.astype(np.float32)
            return v

        values = [get_values(var) for var in tensor_vars.values()]

        self.shapes = [var.shape for var in values]
        self.dtypes = [v.dtype for v in values]
        self.names = list(tensor_vars.keys())
        self.path = path
        self.plain_vars = plain_vars

        if not os.path.exists(path):
            os.mkdir(path)

        with open(_meta_file(path), mode='wb') as file:
            pickle.dump(self, file)

        with tf.Graph().as_default():
            tensor_vars = self._build_vars()
            placeholders = [tf.placeholder(tf.as_dtype(dtype), shape=shape) for dtype, shape in
                            zip(self.dtypes, self.shapes)]
            assigners = [tensor_var.assign(placeholder) for tensor_var, placeholder in
                         zip(tensor_vars, placeholders)]
            feed = {ph: v for ph, v in zip(placeholders, values)}
            saver = tf.train.Saver(self._var_dict(tensor_vars), max_to_keep=1)
            init = tf.global_variables_initializer()

            with tf.Session(config=tf.ConfigProto(device_count={'GPU': 0})) as sess:
                sess.run(init)
                sess.run(assigners, feed_dict=feed)
                save_path = os.path.join(path, 'feeder.cpt')
                saver.save(sess, save_path, write_meta_graph=False, write_state=False)

    def _var_dict(self, variables):
        return {name: var for name, var in zip(self.names, variables)}

    def _build_vars(self):
        def make_tensor(shape, dtype, name):
            tf_type = tf.as_dtype(dtype)
            if tf_type == tf.string:
                empty = ''
            elif tf_type == tf.bool:
                empty = False
            else:
                empty = 0
            init = tf.constant(empty, shape=shape, dtype=tf_type)
            return tf.get_local_variable(name=name, initializer=init, dtype=tf_type)

        with tf.device("/cpu:0"):
            with tf.name_scope('feeder_vars'):
                return [make_tensor(shape, dtype, name) for shape, dtype, name in
                        zip(self.shapes, self.dtypes, self.names)]

    def create_vars(self):
        """
        Builds variable list to use in current graph. Should be called during graph building stage
        :return: variable list with additional restore and create_saver methods
        """
        return FeederVars(self._var_dict(self._build_vars()), self.plain_vars, self.path)

    @staticmethod
    def read_vars(path):
        with open(_meta_file(path), mode='rb') as file:
            feeder = pickle.load(file)
        assert feeder.path == path
        return feeder.create_vars()


class FeederVars(UserDict):
    def __init__(self, tensors: dict, plain_vars: dict, path):
        variables = dict(tensors)
        if plain_vars:
            variables.update(plain_vars)
        super().__init__(variables)
        self.path = path
        self.saver = tf.train.Saver(tensors, name='varfeeder_saver')
        for var in variables:
            if var not in self.__dict__:
                self.__dict__[var] = variables[var]

    def restore(self, session):
        """
        Restores variable content
        :param session: current session
        :return: variable list
        """
        self.saver.restore(session, os.path.join(self.path, 'feeder.cpt'))
        return self

In [42]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Oct  4 12:02:32 2018

@author: vedanshu
"""

import pandas as pd
import numpy as np
import os.path
import os
import argparse

from typing import List

def read_all() -> pd.DataFrame:
    """
    Reads source data for training/prediction
    """
    # Path to cached data
    path = os.path.join('data', 'all.pkl')
    if os.path.exists(path):
        df = pd.read_pickle(path)
        return df
    else:
        state = {0: 'NSW', 1: 'QLD', 2: 'SA', 3: 'TAS', 4: 'VIC'}
        year = {0: '2015', 1: '2016', 2: '2017'}
        
        df_nsw = pd.DataFrame()
        df_qld = pd.DataFrame()
        df_sa = pd.DataFrame()
        df_tas = pd.DataFrame()
        df_vic = pd.DataFrame()
        
        df_all = pd.DataFrame()
        
        df = {'NSW': df_nsw, 'QLD': df_qld, 'SA': df_sa, 'TAS': df_tas, 'VIC': df_vic}
        
        for st in state.values():
            for ye in year.values():
                for mn in range(1,13):
                    if mn < 10:            
                        dataset = pd.read_csv('./data/' + st + '/PRICE_AND_DEMAND_' + ye + '0' + str(mn) +'_' + st + '1.csv')
                    else:
                        dataset = pd.read_csv('./data/' + st + '/PRICE_AND_DEMAND_' + ye + str(mn) +'_' + st + '1.csv')
                    df[st] = df[st].append(dataset.iloc[:,:3])
            df_all = df_all.append(df[st].iloc[:,:])
        df_all = df_all.set_index(['REGION', 'SETTLEMENTDATE'])
        df_all.to_pickle(path)
        return df_all
    
def read_x(state, start, end) -> pd.DataFrame:
    """
    Gets source data from start to end date. Any date can be None
    """
    def read_state(state):
        df = read_all()
        return df.loc[state]
    df = read_state(state)
    if start and end:
        return df.loc[start:end]
    elif end:
        return df.loc[:end]
    else:
        return df

def single_autocorr(series, lag):
    """
    Autocorrelation for single data series
    :param series: traffic series
    :param lag: lag, days
    :return:
    """
    s1 = series[lag:]
    s2 = series[:-lag]
    ms1 = np.mean(s1)
    ms2 = np.mean(s2)
    ds1 = s1 - ms1
    ds2 = s2 - ms2
    divider = np.sqrt(np.sum(ds1 * ds1)) * np.sqrt(np.sum(ds2 * ds2))
    return np.sum(ds1 * ds2) / divider if divider != 0 else 0

def batch_autocorr(series, lag, starts, ends, backoffset=0):
    """
    Calculate autocorrelation for batch (many time series at once)
    :param data: Time series
    :param lag: Autocorrelation lag
    :param starts: Start index for series
    :param ends: End index for series
    :param backoffset: Offset from the series end, days.
    :return: autocorrelation, shape [n_series].
    """
    n_series = series.shape[0]
    max_end = n_series - backoffset
    
    end = min(ends, max_end)
    series = series[starts:end]
    c_365 = single_autocorr(series, lag)
    c_364 = single_autocorr(series, lag-1)
    c_366 = single_autocorr(series, lag+1)
    # Average value between exact lag and two nearest neighborhs for smoothness
    corr = 0.5 * c_365 + 0.25 * c_364 + 0.25 * c_366
    return corr #, support

def prepare_data(start, end, state) -> pd.DataFrame:
    """
    Reads source data, calculates start and end of each series, drops bad series, calculates log1p(series)
    :param start: start date of effective time interval, can be None to start from beginning
    :param end: end date of effective time interval, can be None to return all data
    :param state: state of series
    :return: log1p(series)
    """
    df = read_x(state, start, end)
    return np.log1p(df.fillna(0))

def normalize(values: np.ndarray):
    return (values - values.mean()) / np.std(values)

def lag_indexes(begin, end) -> List[pd.Series]:
    """
    Calculates indexes for 3, 6, 9, 12 months backward lag for the given date range
    :param begin: start of date range
    :param end: end of date range
    :return: List of 4 Series, one for each lag. For each Series, index is date in range(begin, end), value is an index
     of target (lagged) date in a same Series. If target date is out of (begin,end) range, index is -1
    """
    dr = pd.date_range(begin, end, freq='0.5H')
    # key is date, value is day index
    base_index = pd.Series(np.arange(0, len(dr)), index=dr)

    def lag(offset):
        dates = dr - offset
        return pd.Series(data=base_index.loc[dates].fillna(-1).astype(np.int32).values, index=dr)

    return [lag(pd.DateOffset(months=m)) for m in (3, 6, 9, 12)]

def run():
    
    data_dir = "data/vars"
    state = "NSW1"
    add_end = 0
    start = "2015/01/01 00:30:00"
    end = "2018/01/01 00:00:00"
    corr_backoffset = 0
    
    # Get the data
    df = prepare_data(start, end, state)
    
    # Our working date range
    data_start, data_end = pd.to_datetime(df.first_valid_index()), pd.to_datetime(df.last_valid_index())
    
    # We have to project some date-dependent features (day of week, etc) to the future dates for prediction
    features_end = data_end + pd.Timedelta(add_end/2, unit='h')
    print(f"start: {data_start}, end:{data_end}, features_end:{features_end}")
    
    starts = 0
    ends = df.shape[0]
    
    # Yearly(annual) autocorrelation
    year_autocorr = batch_autocorr(df.values, 365, starts, ends, corr_backoffset)
    
    # Quarterly autocorrelation
    quarter_autocorr = batch_autocorr(df.values, int(round(365.25/4)), starts, ends, corr_backoffset)
    
    # Make time-dependent features
    features = pd.date_range(data_start, features_end, freq='0.5H')
    #dow = normalize(features_days.dayofweek.values)
    week_period = 7 / (2 * np.pi)
    dow_norm = features.dayofweek.values / week_period
    dow = np.stack([np.cos(dow_norm), np.sin(dow_norm)], axis=-1)
    
     # Assemble indices for quarterly lagged data
    lagged_ix = np.stack(lag_indexes(data_start, features_end), axis=-1)
    
    # Assemble final output
    tensors = dict(
        hits=df,
        lagged_ix=lagged_ix,
        page_ix=df.index.values,
        dow=dow
        )
    plain = dict(
        features=len(features),
        year_autocorr=year_autocorr,
        quarter_autocorr=quarter_autocorr,
        data_size=df.shape[0],
        data_start=data_start,
        data_end=data_end,
        features_end=features_end
        )

    # Store data to the disk
    VarFeeder(data_dir, tensors, plain)

if __name__ == '__main__':
    """
    python make_features.py data/vars --add_end=0 --start="2015/01/01 00:30:00" --end="2018/01/01 00:00:00"
    """
    run()

start: 2015-01-01 00:30:00, end:2018-01-01 00:00:00, features_end:2018-01-01 00:00:00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:132: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Oct  9 20:09:16 2018

@author: vedanshu
"""

from enum import Enum
import tensorflow as tf

class ModelMode(Enum):
  TRAIN = 0,
  EVAL = 1,
  PREDICT = 2

class InputPipe:
  def __init__(self, inp: VarFeeder, mode: ModelMode, n_epoch=None,
               batch_size=32, runs_in_burst=1, verbose=True, predict_window=48, train_window=36, back_offset=0,
               train_skip_first=0, rand_seed=None):
      """
      Create data preprocessing pipeline
      :param inp: Raw input data
      :param features: Features tensors (subset of data in inp)
      :param mode: Train/Predict/Eval mode selector
      :param n_epoch: Number of epochs. Generates endless data stream if None
      :param batch_size: (https://www.tensorflow.org/guide/datasets#batching_dataset_elements)
      :param n_splits: number of time series split
      :param runs_in_burst: How many batches can be consumed at short time interval (burst). Multiplicator for prefetch() (https://www.tensorflow.org/performance/datasets_performance)
      :param verbose: Print additional information during graph construction
      :param predict_window: Number of days to predict
      :param train_window: Use train_window days for traning
      :param back_offset: Don't use back_offset days at the end of timeseries
      :param train_skip_first: Don't use train_skip_first days at the beginning of timeseries
      :param rand_seed:
      """
      self.inp = inp
      self.batch_size = batch_size
      self.rand_seed = rand_seed
      self.back_offset = back_offset
      self.start_offset = train_skip_first
      self.train_window = train_window
      self.predict_window = predict_window
      self.mode = mode
      self.verbose = verbose
      
      list_half_hourly_load = self.inp.hits[train_skip_first:(-back_offset if back_offset > 0 else None)]
      list_half_hourly_load = list_half_hourly_load / tf.linalg.norm(list_half_hourly_load)
      matrix_hits = list_half_hourly_load[:-(list_half_hourly_load.shape[0].value % (train_window + predict_window))]
      matrix_hits = tf.reshape(matrix_hits, [-1, (train_window + predict_window)])
      
      matrix_dow = self.inp.dow[train_skip_first:(-back_offset if back_offset > 0 else None)]
      matrix_dow = matrix_dow[:-(matrix_dow.shape[0].value % (train_window + predict_window))]
      matrix_dow = tf.reshape(matrix_dow, [-1, (train_window + predict_window), 2])
      
      # Convert -1 to 0 for gather(), it don't accept anything exotic
      lags_indices = tf.maximum(self.inp.lagged_ix, 0)
      # Translate lag indexes to hit values
      lagged_hit = tf.gather(list_half_hourly_load, lags_indices)
      lagged_hit = lagged_hit / tf.linalg.norm(lagged_hit)
      lagged_hit = lagged_hit[train_skip_first:(-back_offset if back_offset > 0 else None)]
      matrix_lagged_hit = lagged_hit[:-(lagged_hit.shape[0].value % (train_window + predict_window))]
      matrix_lagged_hit = tf.reshape(matrix_lagged_hit, [-1, (train_window + predict_window), 4])
      
      x_hits = matrix_hits[:, :train_window]
      y_hits = matrix_hits[:, train_window:]
      
      x_dow = matrix_dow[:, :train_window]
      y_dow = matrix_dow[:, train_window:]
      
      x_lagged_hit = matrix_lagged_hit[:, :train_window]
      y_lagged_hit = matrix_lagged_hit[:, train_window:]

      def tileFeatures(window):
        # [n_features] => [window, n_features]
        flat_features = tf.stack([self.inp.year_autocorr, self.inp.quarter_autocorr], axis=0 )
        features = tf.expand_dims(flat_features, axis=0)
        features = tf.tile(features, [window,1])

        # [train_window, n_features] => [size, window, n_features]
        features = tf.expand_dims(features, axis=0)
        features = tf.tile(features, [x_hits.shape[0].value, 1,1])
        return features
      
      x_features = tf.concat((tf.reshape(x_hits, [x_hits.shape[0].value,train_window,1]),
                                   x_dow, x_lagged_hit, 
                                   tileFeatures(train_window) 
                                   ), axis = 2)
      
      y_features = tf.concat((y_dow, 
                              y_lagged_hit,
                              tileFeatures(predict_window)
                             ), axis = 2)
      
      y_labels = tf.reshape(y_hits, [y_hits.shape[0].value, predict_window, 1])
      
      # Assume that each row of `features` corresponds to the same row as `labels`.
      assert x_features.shape[0] == y_labels.shape[0].value
      
      dataset = tf.data.Dataset.from_tensor_slices((x_features, y_features, y_labels))
      # [Other transformations on `dataset`...]
      #dataset = dataset.shuffle(seed=rand_seed)
      dataset = dataset.repeat(n_epoch)  # Repeat the input indefinitely.
      dataset = dataset.batch(batch_size)
      dataset = dataset.prefetch(runs_in_burst * 2)

      self.iterator = dataset.make_initializable_iterator()
      self.x_feature, self.y_feature, self.y_true = self.iterator.get_next()
      self.encoder_features_depth = self.x_feature.shape[2].value

  def load_vars(self, session):
      self.inp.restore(session)

  def init_iterator(self, session):
      session.run(self.iterator.initializer)


In [0]:
import tensorflow.contrib.training as training

params = dict(
    epochs = 2000,
    batch_size=256,
    train_window=24,
    predict_window=48,
    train_skip_first=0,
    seed = 1,
    rnn_depth=138,
    encoder_readout_dropout=0.6415488109353416,

    encoder_rnn_layers=1,
    decoder_rnn_layers=2,

    decoder_input_dropout=[1.0, 1.0, 1.0],
    decoder_output_dropout=[0.925, 0.925, 1.0],
    decoder_state_dropout=[0.98, 0.98, 0.995],
    decoder_variational_dropout=[False, False, False],
    decoder_candidate_l2=0.0,
    decoder_gates_l2=0.0,
    gate_dropout=0.9275441207192259,
    gate_activation='none',
    encoder_dropout=0.0,
    encoder_stability_loss=0.0,
    encoder_activation_loss=1e-06,
    decoder_stability_loss=0.0,
    decoder_activation_loss=1e-06,
)

def build_hparams():
  return training.HParams(**params)

In [52]:
def mape(y_true, y_pred):
  return tf.reduce_mean(tf.abs(tf.divide(tf.subtract(y_pred, y_true),(y_true + 1e-10))))

hparams = build_hparams()
tf.reset_default_graph()
inp = VarFeeder.read_vars("data/vars")
pipe_train = InputPipe(inp, mode=ModelMode.TRAIN, batch_size=hparams.batch_size, 
                       n_epoch=hparams.epochs, verbose=False, 
                       train_window=hparams.train_window, predict_window=hparams.predict_window,
                       rand_seed=hparams.seed)

def build_rnn():
  return tf.contrib.cudnn_rnn.CudnnGRU(num_layers=hparams.encoder_rnn_layers, num_units=hparams.rnn_depth,
             direction='unidirectional',
             dropout=hparams.encoder_dropout, seed=hparams.seed)

cuda_model = build_rnn()

rnn_time_input = pipe_train.x_feature

rnn_out, (rnn_state,) = cuda_model(inputs=rnn_time_input)

learning_rate = 0.001   #small learning rate so we don't overshoot the minimum

fc_input = tf.reshape(rnn_out, [-1, hparams.train_window*hparams.rnn_depth])
fc_outputs = tf.layers.dense(fc_input, hparams.predict_window, 
                             kernel_initializer=tf.zeros_initializer(),
                            activation = tf.tanh)       #specify the type of layer (dense)
outputs = tf.reshape(fc_outputs, [-1, hparams.predict_window, 1])          #shape of results

loss = tf.reduce_mean(tf.square(outputs - pipe_train.y_true))    #define the cost function which evaluates the quality of our model
mape_loss = mape(pipe_train.y_true, outputs)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)          #gradient descent method
training_op = optimizer.minimize(loss)          #train the result of the application of the cost_function                                 

model_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
# Make EMA object and update interal variables after optimization step
ema = tf.train.ExponentialMovingAverage(decay=0.9999)
with tf.control_dependencies([training_op]):
    train_op = ema.apply(model_vars)

# Transfer EMA values to original variables
retrieve_ema_weights_op = tf.group([tf.assign(var, ema.average(var)) for var in model_vars])

init = tf.global_variables_initializer()           #initialize all the variables

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    init.run()
    inp.restore(sess)
    pipe_train.init_iterator(sess)

    for ep in range(hparams.epochs):
        _, error = sess.run([training_op, mape_loss])
        if ep > 1000:
          # Copy EMA values to weights
          sess.run(retrieve_ema_weights_op)
        if ep % 100 == 0:
            print(ep, "MAPE:", error*100)
print("\n")


INFO:tensorflow:Restoring parameters from data/vars/feeder.cpt
0 MAPE: 100.0
100 MAPE: 13.244129717350006
200 MAPE: 3.5378877073526382
300 MAPE: 2.057003602385521
400 MAPE: 1.6309591010212898
500 MAPE: 2.246139384806156
600 MAPE: 2.732541970908642
700 MAPE: 1.8203072249889374
800 MAPE: 2.3227766156196594
900 MAPE: 1.9006291404366493
1000 MAPE: 1.7891338095068932
1100 MAPE: 100.0
1200 MAPE: 100.0
1300 MAPE: 100.0
1400 MAPE: 100.0


KeyboardInterrupt: ignored